### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  # (what's wrong with identation?)
  # Check if the lightcurve contains enough data points after sigma clipping
  if len(mag) < 100:
      print(f"\n ################################################## Too many lightcurve points removed by sigma clipping\n")
      return
    

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.write("color_string =\n")
  output_file.write("visual_inspection_comments =\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [6]:
#folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"
#folder_path = "/home/kirill/variablestars/variable_star_classification/downloaded_lightcurves/"

zoe_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"
kirill_path = "/home/kirill/variablestars/variable_star_classification/downloaded_lightcurves/"

if os.path.exists(zoe_path):
    folder_path = zoe_path
elif os.path.exists(kirill_path):
    folder_path = kirill_path
else:
    raise Exception("Neither of the specified directories exist.")

print(f"Using folder path: {folder_path}")

Using folder path: /home/kirill/variablestars/variable_star_classification/downloaded_lightcurves/


In [ ]:
#num_iterations = 1 # Why do we need it?
empty_list = []
#for i in range(7900, 8900):
for i in range(10000, 11000):

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    # Check if the length of web_format is less than 100
    if len(web_format) < 100:
        print(f"\n ################################################## Skipping index {i} due to insufficient data (length: {len(web_format)})\n")
        continue
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)
    
    try:
        classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])
    except Exception as e:
        print(f"\n ################################################## Error in classification_model for index {i}: {str(e)}\n")
        continue

Pulled 1 of 1


######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.79


######### Lightcurve features #########

amplitude = 0.2649477905421564
cusum = 0.1809104317271198
eta = 2.0619407386892994
hl_amp_ratio = 1.2537693481771621
kurtosis = -1.4077801531084364
n_points = 745
period = 0.34177826558390617
period_SNR = 208.71830867529243
period_log10FAP = -134.948638462324
period_uncertainty = 4.1798536162263034e-05
phase_cusum = 0.3735927576119566
phase_eta = 0.2973684377554926
phi21 = 2.0907957870980214
phi31 = 4.083868998918644
quartile31 = 0.38546551941393936
r21 = 0.08913028742473889
r31 = 0.0814122250386692
shapiro_w = 0.924217939376831
skewness = 0.13297731233879698
slope_per10 = -0.04890780327892649
slope_per90 = 0.05583580068999473
stetson_k = 0.8930149190781264
weighted_mean = 14.381057114741816
weighted_std = 0.18164843489164684
Pulled 1 of 1


######### Classification results #########

f